In [4]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
![ ! -e "$(basename spark-2.4.1-bin-hadoop2.7.tgz)" ] && wget  http://apache.osuosl.org/spark/spark-2.4.1/spark-2.4.1-bin-hadoop2.7.tgz
!tar xf spark-2.4.1-bin-hadoop2.7.tgz

--2019-04-09 12:28:17--  http://apache.osuosl.org/spark/spark-2.4.1/spark-2.4.1-bin-hadoop2.7.tgz
Resolving apache.osuosl.org (apache.osuosl.org)... 64.50.233.100, 64.50.236.52, 140.211.166.134, ...
Connecting to apache.osuosl.org (apache.osuosl.org)|64.50.233.100|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 230778742 (220M) [application/x-gzip]
Saving to: ‘spark-2.4.1-bin-hadoop2.7.tgz’

spark-2.4.1-bin-had 100%[===================>] 220.09M  24.6MB/s    in 7.9s    

2019-04-09 12:28:25 (28.0 MB/s) - ‘spark-2.4.1-bin-hadoop2.7.tgz’ saved [230778742/230778742]



In [7]:
!pip install -q findspark

! [ ! -e "$(basename ss.csv)" ] && wget  https://raw.githubusercontent.com/borodark/wsu/master/platforms/assignment_4/ss.csv

! [ ! -e "$(basename ch.csv)" ] && wget  https://raw.githubusercontent.com/borodark/wsu/master/platforms/assignment_4/ch.csv

--2019-04-09 12:35:16--  https://raw.githubusercontent.com/borodark/wsu/master/platforms/assignment_4/ch.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 213061 (208K) [text/plain]
Saving to: ‘ch.csv’

ch.csv              100%[===================>] 208.07K  --.-KB/s    in 0.02s   

2019-04-09 12:35:17 (9.43 MB/s) - ‘ch.csv’ saved [213061/213061]



In [0]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-2.4.1-bin-hadoop2.7"

In [0]:
import findspark
findspark.init()
from pyspark.sql import SparkSession

# get a spark session. 
spark = SparkSession.builder.master("local[*]").getOrCreate()

In [15]:

ss_ = spark.read.csv("ss.csv", header= True, 
                      inferSchema = True)

print(ss_.columns)

columns_lambda = lambda k: k.endswith(', Current week')

common_cols = ['Reporting Area','MMWR Year','MMWR Week']

ss_cols = list(filter(columns_lambda, ss_.columns))


to_keep = common_cols + ss_cols

dfss = ss_.select(*to_keep).withColumnRenamed('Reporting Area', 'area').withColumnRenamed('MMWR Year','year').withColumnRenamed('MMWR Week','week')
dfss.columns

['Reporting Area', 'MMWR Year', 'MMWR Week', 'Salmonellosis (excluding Paratyphoid fever andTyphoid fever)†, Current week', 'Salmonellosis (excluding Paratyphoid fever andTyphoid fever)†, Current week, flag', 'Salmonellosis (excluding Paratyphoid fever andTyphoid fever)†, Previous 52 weeks Med', 'Salmonellosis (excluding Paratyphoid fever andTyphoid fever)†, Previous 52 weeks Med, flag', 'Salmonellosis (excluding Paratyphoid fever andTyphoid fever)†, Previous 52 weeks Max', 'Salmonellosis (excluding Paratyphoid fever andTyphoid fever)†, Previous 52 weeks Max, flag', 'Salmonellosis (excluding Paratyphoid fever andTyphoid fever)†, Cum 2018', 'Salmonellosis (excluding Paratyphoid fever andTyphoid fever)†, Cum 2018, flag', 'Salmonellosis (excluding Paratyphoid fever andTyphoid fever)†, Cum 2017', 'Salmonellosis (excluding Paratyphoid fever andTyphoid fever)†, Cum 2017, flag', 'Shiga toxin-producing Escherichia coli, Current week', 'Shiga toxin-producing Escherichia coli, Current week, flag

['area',
 'year',
 'week',
 'Salmonellosis (excluding Paratyphoid fever andTyphoid fever)†, Current week',
 'Shiga toxin-producing Escherichia coli, Current week',
 'Shigellosis, Current week']

In [16]:
ch_ = spark.read.csv("ch.csv", header= True, 
                      inferSchema = True)

ch_cols = list(filter(columns_lambda, ch_.columns))


to_keep = common_cols + ch_cols

chss = ch_.select(*to_keep).withColumnRenamed('Reporting Area', 'area').withColumnRenamed('MMWR Year','year').withColumnRenamed('MMWR Week','week')

chss.columns

['area',
 'year',
 'week',
 'Chlamydia trachomatis infection, Current week',
 'Coccidioidomycosis, Current week']

In [24]:
to_keep = [chss.area, chss.year, chss.week] + ss_cols + ch_cols

chss = chss.join(dfss, [chss.area == dfss.area, chss.week == dfss.week ,chss.year == dfss.year]).select(*to_keep)
chss.show(100)
chss.printSchema()

+--------------+----+----+---------------------------------------------------------------------------+----------------------------------------------------+-------------------------+---------------------------------------------+--------------------------------+
|          area|year|week|Salmonellosis (excluding Paratyphoid fever andTyphoid fever)†, Current week|Shiga toxin-producing Escherichia coli, Current week|Shigellosis, Current week|Chlamydia trachomatis infection, Current week|Coccidioidomycosis, Current week|
+--------------+----+----+---------------------------------------------------------------------------+----------------------------------------------------+-------------------------+---------------------------------------------+--------------------------------+
| UNITED STATES|2018|   1|                                                                          9|                                                  15|                       50|                                    